In [1]:
import pandas as pd
import re
pd.options.display.max_colwidth = 200

In [2]:
from pandas_profiling import ProfileReport
df = pd.read_excel("../data/raw/Dg from report.xlsx")
df.head(10)

,Patient,Date,mainDgCode,OtherDgCode
0,209536,2015-01-29,E06.3: Autoimunitní tyroiditida,E78.0: Čistá hypercholesterolemie
1,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",Autoimunní thyreoiditis
2,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",Perniciózní anemie
3,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem","Surditas vlevo, tinnitus vlevo"
4,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",Difuzní jaterní léze - steatóza až incipientní fibróza
5,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",Chronická atrofická gastritida typu A
6,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem","Polyvalentní alergie - potraviny, pyly, kyselina acetylsalicylová"
7,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem","Vertebrogenní algický syndrom při protruzi až herniaci plotének L3/4, L4/5, L5/S1"
8,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem","Reaktivní depresivní sy, t.č. bez medikace"
9,8451,2015-01-08,"E10.9: Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",Galukom


In [4]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report")
profile.to_file("diag_orig.html")

C:\Users\pavel\anaconda3\envs\healthack\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Summarize dataset:  38%|███████████████████████████▊                                              | 6/16 [00:05<00:14,  1.41s/it, Calculate spearman correlation]C:\Users\pavel\anaconda3\envs\healthack\lib\site-packages\multimethod\__init__.py:315: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)
Export report to file: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 215.56it/s]


In [4]:
# Split Codes and Descriptions whenever possible
df["mainDgCode"] = df["mainDgCode"].str.strip()
df["OtherDgCode"] = df["OtherDgCode"].str.strip()
df[["main_code", "main_description"]] = df["mainDgCode"].str.split(":", n = 1, expand = True)
condition = df["OtherDgCode"].str.match("[A-Z]\d", na = False)
df.loc[condition,  ["other_code", "other_description"]] = df.loc[condition, "OtherDgCode"].str.split(":", n = 1, expand = True).values
df.loc[~condition, "other_description"] = df.loc[~condition, "OtherDgCode"] 
df

# Prepare target variable
df["is_ckd"] = df["main_code"].str.startswith("N18", na = False) \
                | df["main_description"].str.lower().str.contains("chronické onemocnění ledvin", na = False) \
                | df["other_code"].str.startswith("N18", na = False) \
                | df["other_description"].str.lower().str.contains("chronické onemocnění ledvin", na = False)
df["is_ckd"] = df["is_ckd"].astype(int)
df["is_dia"] = df["main_description"].str.lower().str.contains("diabetes mellitus", na = False) \
                | df["other_description"].str.lower().str.contains("diabetes mellitus", na = False)
df["is_dia"] = df["is_dia"].astype(int)

In [5]:
# Add data about stadium of ckd from main and other diagnosis
df["ckd_stadium"] = df.loc[df["is_ckd"] == 1, "mainDgCode"].str.extract(r'chronické onemocnění ledvin, stadium (\d)', re.IGNORECASE, expand=True)
df.loc[df["ckd_stadium"].isna(), "ckd_stadium"] = df.loc[df["ckd_stadium"].isna(), "OtherDgCode"].str.extract(r'chronické onemocnění ledvin, stadium (\d)', re.IGNORECASE, expand=True).values
# display 
df.loc[(~df["ckd_stadium"].isna()) & (df["other_code"].str.contains("N18")) , :]

,Patient,Date,mainDgCode,OtherDgCode,main_code,main_description,other_code,other_description,is_ckd,is_dia,ckd_stadium
58432,1179284,2022-01-06,E13.9: Jiný určený diabetes mellitus bez komplikací (potransplantační diabetes),"N18.3: Chronické onemocnění ledvin, stadium 3, nefrotický sy",E13.9,Jiný určený diabetes mellitus bez komplikací (potransplantační diabetes),N18.3,"Chronické onemocnění ledvin, stadium 3, nefrotický sy",1,1,3
58451,10721,2022-01-12,"E10.7: Diabetes mellitus 1. typu od 1986, s mnohočetnými komplikacemi, nedostatečně kompenzovaný inzulinem podávaným pumpou (HbA1c 74, 4/2013)","N18.3: Chronické onemocnění ledvin, stadium 3",E10.7,"Diabetes mellitus 1. typu od 1986, s mnohočetnými komplikacemi, nedostatečně kompenzovaný inzulinem podávaným pumpou (HbA1c 74, 4/2013)",N18.3,"Chronické onemocnění ledvin, stadium 3",1,1,3
58781,17774,2022-02-17,E11.6: Diabetes mellitus nezávislý na inzulinu - s jinými určenými komplikacemi,"N18.1: Chronické onemocnění ledvin, stadium 1",E11.6,Diabetes mellitus nezávislý na inzulinu - s jinými určenými komplikacemi,N18.1,"Chronické onemocnění ledvin, stadium 1",1,1,1
60101,13096,2022-01-04,"I48.2: Permanentní fibrilace síní, CHA2DS2VASc:5, rivaroxaban","N18.3: Chronické onemocnění ledvin, stadium 3",I48.2,"Permanentní fibrilace síní, CHA2DS2VASc:5, rivaroxaban",N18.3,"Chronické onemocnění ledvin, stadium 3",1,0,3
60933,196834,2022-01-05,"I50.0: Městnavé selhání srdce, NYHA II","N18.2: Chronické onemocnění ledvin, stadium 2",I50.0,"Městnavé selhání srdce, NYHA II",N18.2,"Chronické onemocnění ledvin, stadium 2",1,0,2
...,...,...,...,...,...,...,...,...,...,...,...
138209,131277,2022-11-22,E105: Sy diabetické nohy v ischemickém terénu,"N18.3: Chronické onemocnění ledvin, stadium 3",E105,Sy diabetické nohy v ischemickém terénu,N18.3,"Chronické onemocnění ledvin, stadium 3",1,0,3
138351,154315,2022-11-23,E10.7: Diabetes mellitus 1. typu s mnohočetnými komplikacemi,"N18.2: Chronické onemocnění ledvin, stadium 2, DM nefropathie",E10.7,Diabetes mellitus 1. typu s mnohočetnými komplikacemi,N18.2,"Chronické onemocnění ledvin, stadium 2, DM nefropathie",1,1,2
138495,1200860,2022-11-23,"I36.1: Význ. TriR (4/4) při dilataci anulu, dilat. PK s hran. funkcí, na 4 cípé trikusp. chlopni , Triclip 2/2022, rezzultuje významná TRiR","N18.2: Chronické onemocnění ledvin, stadium 2",I36.1,"Význ. TriR (4/4) při dilataci anulu, dilat. PK s hran. funkcí, na 4 cípé trikusp. chlopni , Triclip 2/2022, rezzultuje významná TRiR",N18.2,"Chronické onemocnění ledvin, stadium 2",1,0,2
138659,180355,2022-11-24,E11.8: Diabetes mellitus 2. typu s neurčenými komplikacemi na PAD,"N18.3: Chronické onemocnění ledvin, stadium 3 CKDG3b A2 dle KDIGO, afunkční PL , kde 2005 hydronefróza se stenózou ústí P ureteru",E11.8,Diabetes mellitus 2. typu s neurčenými komplikacemi na PAD,N18.3,"Chronické onemocnění ledvin, stadium 3 CKDG3b A2 dle KDIGO, afunkční PL , kde 2005 hydronefróza se stenózou ústí P ureteru",1,1,3


In [6]:
df_grouped = df.groupby(["Patient"]).agg(min_date = pd.NamedAgg("Date", "min"),
                           max_date = pd.NamedAgg("Date", "max"),
                           cnt_rows = pd.NamedAgg("Date", "count"),
                           cnt_visits = pd.NamedAgg("Date", "nunique"),
                           is_ckd_patient = pd.NamedAgg("is_ckd", "max"),
                           is_dia_patient = pd.NamedAgg("is_dia", "max")).sort_values("cnt_rows")
df_grouped_ckd = df.loc[df["is_ckd"] == 1, :].groupby("Patient").agg(min_ckd_date = pd.NamedAgg("Date", "min")).reset_index()
df_grouped_dia = df.loc[df["is_dia"] == 1, :].groupby("Patient").agg(min_dia_date = pd.NamedAgg("Date", "min")).reset_index()
df_grouped["date_range"] = df_grouped["max_date"] - df_grouped["min_date"]
df_grouped = df_grouped.reset_index()\
    .merge(df_grouped_ckd, on = "Patient", how = "left")\
    .merge(df_grouped_dia, on ="Patient", how = "left")
df_joined = df.merge(df_grouped, on = "Patient", how = "inner")
df_joined.loc[~df_joined["min_dia_date"].isna(), :]
df_joined["is_ckd_cum"] = (df_joined["min_ckd_date"] <= df_joined["Date"]).astype(int)
df_joined["is_dia_cum"] = (df_joined["min_ckd_date"] <= df_joined["Date"]).astype(int)
df_joined["is_ckd_cum"].fillna(0, inplace=True)
df_joined["is_dia_cum"].fillna(0, inplace=True)
df_joined.loc[df_joined["Patient"] == 30877, :].sort_values("Date")

,Patient,Date,mainDgCode,OtherDgCode,main_code,main_description,other_code,other_description,is_ckd,is_dia,...,max_date,cnt_rows,cnt_visits,is_ckd_patient,is_dia_patient,date_range,min_ckd_date,min_dia_date,is_ckd_cum,is_dia_cum
494,30877,2015-01-15,E06.3: Autoimunitní tyroiditida s uzlovou přestavbou,Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie),E06.3,Autoimunitní tyroiditida s uzlovou přestavbou,NaN,Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie),0,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,0,0
495,30877,2015-01-15,E06.3: Autoimunitní tyroiditida s uzlovou přestavbou,I15.0: Renovaskulární hypertenze,E06.3,Autoimunitní tyroiditida s uzlovou přestavbou,I15.0,Renovaskulární hypertenze,0,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,0,0
496,30877,2015-01-15,E06.3: Autoimunitní tyroiditida s uzlovou přestavbou,E79.0: Hyperurikemie bez známek zánětlivé artritidy a tofů,E06.3,Autoimunitní tyroiditida s uzlovou přestavbou,E79.0,Hyperurikemie bez známek zánětlivé artritidy a tofů,0,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,0,0
497,30877,2015-01-15,E06.3: Autoimunitní tyroiditida s uzlovou přestavbou,E78.0: Čistá hypercholesterolemie,E06.3,Autoimunitní tyroiditida s uzlovou přestavbou,E78.0,Čistá hypercholesterolemie,0,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,0,0
498,30877,2015-01-15,E06.3: Autoimunitní tyroiditida s uzlovou přestavbou,St.p. HYE a adnexotomii pro Ca dělož čípku,E06.3,Autoimunitní tyroiditida s uzlovou přestavbou,NaN,St.p. HYE a adnexotomii pro Ca dělož čípku,0,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,0,0
499,30877,2015-01-15,E06.3: Autoimunitní tyroiditida s uzlovou přestavbou,E11.9: Diabetes mellitus 2. typu bez komplikací,E06.3,Autoimunitní tyroiditida s uzlovou přestavbou,E11.9,Diabetes mellitus 2. typu bez komplikací,0,1,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,0,0
500,30877,2022-01-25,"N18.9: Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",- stp. oboustranné PTA renálních tepen a zavedení stentů (2000),N18.9,"Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",NaN,- stp. oboustranné PTA renálních tepen a zavedení stentů (2000),1,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,1,1
501,30877,2022-01-25,"N18.9: Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",E11.9: Diabetes mellitus 2. typu bez komplikací léčen dietou,N18.9,"Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",E11.9,Diabetes mellitus 2. typu bez komplikací léčen dietou,1,1,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,1,1
502,30877,2022-01-25,"N18.9: Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",Autoimunitní tyroiditida s uzlovou přestavbou,N18.9,"Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",NaN,Autoimunitní tyroiditida s uzlovou přestavbou,1,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,1,1
503,30877,2022-01-25,"N18.9: Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",Osteopenie,N18.9,"Chronická renální insuficience s hypertenzí renovaskulární etiologie (v.s. na podkladě fibromuskulární dysplazie), CKD 1",NaN,Osteopenie,1,0,...,2022-01-25,16,2,1,1,2567 days,2022-01-25,2015-01-15,1,1


In [7]:
#rename some of the columns
df_joined.rename(columns = {"Patient": "patient_id", "Date": "date"}, inplace=True)
df_joined.head()

,patient_id,date,mainDgCode,OtherDgCode,main_code,main_description,other_code,other_description,is_ckd,is_dia,...,max_date,cnt_rows,cnt_visits,is_ckd_patient,is_dia_patient,date_range,min_ckd_date,min_dia_date,is_ckd_cum,is_dia_cum
0,209536,2015-01-29,E06.3: Autoimunitní tyroiditida,E78.0: Čistá hypercholesterolemie,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,...,2022-06-01,5,5,0,0,2680 days,NaT,NaT,0,0
1,209536,2015-08-06,Z01.4: Gynekologické vyšetření (všeobecné)(rutinní),N83.1: Cysta žlutého tělíska,Z01.4,Gynekologické vyšetření (všeobecné)(rutinní),N83.1,Cysta žlutého tělíska,0,0,...,2022-06-01,5,5,0,0,2680 days,NaT,NaT,0,0
2,209536,2022-01-10,E06.3: Autoimunitní tyroiditida,E78.0: Čistá hypercholesterolemie,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,...,2022-06-01,5,5,0,0,2680 days,NaT,NaT,0,0
3,209536,2022-04-04,D25.1: Intramurální leiomyom dělohy,NaN,D25.1,Intramurální leiomyom dělohy,NaN,NaN,0,0,...,2022-06-01,5,5,0,0,2680 days,NaT,NaT,0,0
4,209536,2022-06-01,Z01.4: Gynekologické vyšetření (všeobecné)(rutinní),D25.1: Intramurální leiomyom dělohy,Z01.4,Gynekologické vyšetření (všeobecné)(rutinní),D25.1,Intramurální leiomyom dělohy,0,0,...,2022-06-01,5,5,0,0,2680 days,NaT,NaT,0,0


In [ ]:
# IS chronické onemocnění ledvin G4A1 - CKD

In [8]:
for col in ["main_code", "main_description", "other_code", "other_description"]:
    df_joined[col] = df_joined[col].str.strip()

In [9]:
#TODO: Add Diabetes
df_joined.drop(columns = ["mainDgCode","OtherDgCode"])\
    .to_csv("../data/preprocessed/diagnoses.csv", index=False, sep=";")

In [11]:
x = pd.read_csv("../data/preprocessed/diagnoses.csv", sep = ";")

In [13]:
x

,patient_id,date,main_code,main_description,other_code,other_description,is_ckd,is_dia,ckd_stadium,min_date,max_date,cnt_rows,cnt_visits,is_ckd_patient,is_dia_patient,date_range,min_ckd_date,min_dia_date,is_ckd_cum,is_dia_cum
0,209536,2015-01-29,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,NaN,2015-01-29,2022-06-01,5,5,0,0,2680 days,NaN,NaN,0,0
1,209536,2015-08-06,Z01.4,Gynekologické vyšetření (všeobecné)(rutinní),N83.1,Cysta žlutého tělíska,0,0,NaN,2015-01-29,2022-06-01,5,5,0,0,2680 days,NaN,NaN,0,0
2,209536,2022-01-10,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,NaN,2015-01-29,2022-06-01,5,5,0,0,2680 days,NaN,NaN,0,0
3,209536,2022-04-04,D25.1,Intramurální leiomyom dělohy,NaN,NaN,0,0,NaN,2015-01-29,2022-06-01,5,5,0,0,2680 days,NaN,NaN,0,0
4,209536,2022-06-01,Z01.4,Gynekologické vyšetření (všeobecné)(rutinní),D25.1,Intramurální leiomyom dělohy,0,0,NaN,2015-01-29,2022-06-01,5,5,0,0,2680 days,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138713,1217771,2015-11-11,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",K20,Zánět jícnu - ezofagitida,0,0,NaN,2015-11-11,2022-07-29,5,4,0,0,2452 days,NaN,NaN,0,0
138714,1217771,2022-07-29,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",I10,Esenciální (primární) hypertenze,0,0,NaN,2015-11-11,2022-07-29,5,4,0,0,2452 days,NaN,NaN,0,0
138715,1217771,2022-07-29,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",NaN,Hypercholesterolemie,0,0,NaN,2015-11-11,2022-07-29,5,4,0,0,2452 days,NaN,NaN,0,0
138716,1217771,2022-03-25,K50.8,"Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",NaN,NaN,0,0,NaN,2015-11-11,2022-07-29,5,4,0,0,2452 days,NaN,NaN,0,0
